In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import heapq # for minimum spanning tree
import pickle # for saving the data
import os
import itertools

In [22]:
# from Prof. Jose's code
def returns_to_covar(returns):
    num_col = skips
    test = np.zeros(shape = (num_col, num_col))
    
    for i, j in list(itertools.combinations(np.arange(num_col), 2)):
        row_mask = (~np.isnan(returns[:, i]))&(~np.isnan(returns[:, j]))
        pi = returns[:, i][row_mask]
        pj = returns[:, j][row_mask]
        test[i, j] = np.sum((pi - np.mean(pi))*(pj - np.mean(pj)))/len(pi)
        #print(i, j)
        
    test = np.where(test,test,test.T)
    for i in range(num_col):
        test[i, i] = np.nanstd(returns[:, i])**2
    
    return test

# from: https://gist.github.com/wiso/ce2a9919ded228838703c1c7c7dad13b
def covar_to_correl(cov_matrix):
    std_devs = np.sqrt(np.diag(cov_matrix))
    outer_v = np.outer(std_devs, std_devs)
    correlation = cov_matrix / outer_v
    correlation[cov_matrix == 0] = 0
    return correlation

# from: https://stackoverflow.com/questions/42908334/checking-if-a-matrix-is-symmetric-in-numpy
# def check_symmetric(a, rtol=1e-05, atol=1e-08):
#     return np.allclose(a, a.T, rtol=rtol, atol=atol)

def check_symmetric(a, tol=1e-8):
    return np.all(np.abs(a-a.T) < tol)

## own dataframe

In [15]:
test_df = pd.DataFrame({
    'a':[1, 2, 3, 4, 5, 6, 6, 5, 4, 3, 2, 1, 2, 2, 2, 2, 2, 2],
    'b':[1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6],
    'c':[np.nan, 1, 9, np.nan, 1, 9, np.nan, 1, 9, np.nan, 1, 9, np.nan, 1, 9, np.nan, 1, 9]
},
    index=pd.date_range(20200601, periods=18, freq='d')
)
test_arr = test_df.to_numpy()
test_arr

array([[ 1.,  1., nan],
       [ 2.,  2.,  1.],
       [ 3.,  3.,  9.],
       [ 4.,  4., nan],
       [ 5.,  5.,  1.],
       [ 6.,  6.,  9.],
       [ 6.,  1., nan],
       [ 5.,  2.,  1.],
       [ 4.,  3.,  9.],
       [ 3.,  4., nan],
       [ 2.,  5.,  1.],
       [ 1.,  6.,  9.],
       [ 2.,  1., nan],
       [ 2.,  2.,  1.],
       [ 2.,  3.,  9.],
       [ 2.,  4., nan],
       [ 2.,  5.,  1.],
       [ 2.,  6.,  9.]])

In [16]:
skips = len(test_df.columns)
window = 4
counter = 0

for i in range(len(test_df.index)):
    # this is to get the matrix
    returns = test_arr[i:i+window]

    # this is to get the covar matrix
    covar_matrix = returns_to_covar(returns)
    counter += 1
    # print(f"Covar matrix:\n{covar_matrix}")
    
    # this is to get the correl matrix
    correl_matrix = covar_to_correl(covar_matrix)
    # print(f"Correl matrix:\n{correl_matrix}")
    
    # this is to get the distance matrix
    distance_matrix = 1 - np.square(correl_matrix)
    # print(f"Distance matrix:\n{distance_matrix}")

    matrices = [covar_matrix, correl_matrix, distance_matrix]
    for matrix in matrices:
        if check_symmetric(matrix) == False:
            print("This matrix is not symmetrical.")
            print(counter)

/tmp/ipykernel_16150/651759837.py:23: RuntimeWarning: invalid value encountered in divide
  correlation = cov_matrix / outer_v


In [5]:
# skips = len(test_df.columns)
# window = 55

# for i in range(len(test_df.index)):
#     # this is to get the matrix
#     returns = test_arr[i:i+window]

#     # this is to get the covar matrix
#     covar_matrix = returns_to_covar(returns)
#     print(covar_matrix)
    
#     # this is to make the correlation matrix 
#     correl_matrix = covar_to_correl(covar_matrix) 
    
#     # this is to make the distance matrix
#     distance_matrix = 1 - np.square(correl_matrix)

#     # this is to save it into a pickle file
#     tables = {"covariance matrix": covar1, 
#               "correlation matrix": correl_matrix, 
#               "distance matrix": distance_matrix}
    
#     # folder_path = "matrices"
#     # name = covar1.index.get_level_values("date_id").unique().to_list()[0].strftime("%Y-%m-%d")
#     # file_path = os.path.join(folder_path, f"{name}.pickl")

#     # pd.to_pickle(tables, file_path)

## 3 stocks in the PSE

In [12]:
PSE_data = pd.read_csv(
    "daily.csv"
)
PSE_data['date_id'] = pd.to_datetime(PSE_data['date_id'])
PSE_data = PSE_data.set_index("date_id")
df_filtered = PSE_data.drop(columns=[col for col in PSE_data.columns if not (col.startswith('PH_') and col.endswith('_P')) and col != 'date_id'])

df_filtered = df_filtered.loc['2013-06-25':'2023-09-26']

def has_high_null_percentage(column, null_threshold):
    return column.isnull().mean() > null_threshold

null_percentage_threshold = round(df_filtered.shape[0] * 0.85)
columns_to_drop = []

for col in df_filtered.columns:
    if col == "date_id":  
        continue
    if has_high_null_percentage(df_filtered[col], null_percentage_threshold):
        columns_to_drop.append(col)

df_filtered = df_filtered.drop(columns=columns_to_drop)

PSE_log_returns = df_filtered.copy()
numeric_cols = PSE_log_returns.columns.difference(["date_id"])
PSE_log_returns[numeric_cols] = PSE_log_returns[numeric_cols].replace(0, np.nan)
PSE_log_returns[numeric_cols] = np.log(PSE_log_returns[numeric_cols] / PSE_log_returns[numeric_cols].shift(1))

PSE_log_returns = PSE_log_returns.loc['2013-06-26':'2023-09-26']
PSE_log_returns.columns = PSE_log_returns.columns.str.replace(r"_P", "").str.replace(r"PH_", "")

# this was for testing
least_null_cols = PSE_log_returns.isna().sum().nsmallest(3).index
trial = PSE_log_returns[least_null_cols]
trial

,ASA,ABS,AGN
date_id,,,
2013-06-26,0.154151,0.026668,0.182322
2013-06-27,0.000000,0.025975,0.000000
2013-06-28,0.000000,-0.025975,0.000000
2013-07-01,0.000000,0.051293,0.000000
2013-07-02,0.000000,0.000000,0.000000
...,...,...,...
2023-09-20,0.000000,0.000000,0.000000
2023-09-21,0.000000,0.000000,0.000000
2023-09-22,0.000000,0.000000,0.000000


In [13]:
trial_returns = trial.copy()
PSE_log_returns = trial.to_numpy()

In [14]:
skips = len(trial_returns.columns)
window = 110
counter = 0

for i in range(len(trial_returns.index)):
    # this is to get the matrix
    returns = PSE_log_returns[i:i+window]

    # this is to get the covar matrix
    covar_matrix = returns_to_covar(returns)
    counter += 1
    # print(f"Covar matrix:\n{covar_matrix}")
    
    # this is to get the correl matrix
    correl_matrix = covar_to_correl(covar_matrix)
    # print(f"Correl matrix:\n{correl_matrix}")
    
    # this is to get the distance matrix
    distance_matrix = 1 - np.square(correl_matrix)
    # print(f"Distance matrix:\n{distance_matrix}")

    matrices = [covar_matrix, correl_matrix, distance_matrix]
    for matrix in matrices:
        if check_symmetric(matrix) == False:
            print("This matrix is not symmetrical.")
            print(counter)

/tmp/ipykernel_16150/651759837.py:23: RuntimeWarning: invalid value encountered in divide
  correlation = cov_matrix / outer_v


## Gaussian distribution

### This is for a 102 x 3 array

In [6]:
np.random.seed(42)
data = np.random.randn(102, 3)

np.random.seed(42)
nan_mask = np.random.rand(102, 3) < 0.1  # 10% missing values
data[nan_mask] = np.nan
data

array([[ 0.49671415, -0.1382643 ,  0.64768854],
       [ 1.52302986, -0.23415337, -0.23413696],
       [        nan,  0.76743473, -0.46947439],
       [ 0.54256004,         nan, -0.46572975],
       [ 0.24196227, -1.91328024, -1.72491783],
       [-0.56228753, -1.01283112,  0.31424733],
       [-0.90802408, -1.4123037 ,  1.46564877],
       [-0.2257763 ,  0.0675282 , -1.42474819],
       [-0.54438272,  0.11092259, -1.15099358],
       [ 0.37569802, -0.60063869,         nan],
       [-0.60170661,  1.85227818,         nan],
       [-1.05771093,  0.82254491, -1.22084365],
       [ 0.2088636 ,         nan, -1.32818605],
       [ 0.19686124,  0.73846658,  0.17136828],
       [        nan, -0.3011037 , -1.47852199],
       [-0.71984421, -0.46063877,  1.05712223],
       [ 0.34361829, -1.76304016,  0.32408397],
       [-0.38508228, -0.676922  ,  0.61167629],
       [ 1.03099952,  0.93128012,         nan],
       [-0.30921238,         nan,  0.97554513],
       [-0.47917424, -0.18565898, -1.106

In [7]:
skips = 3
window = 10
counter = 0

for i in range(len(data)):
    # this is to get the matrix
    returns = data[i:i+window]

    # this is to get the covar matrix
    covar_matrix = returns_to_covar(returns)
    counter += 1
    # print(f"Covar matrix:\n{covar_matrix}")
    
    # this is to get the correl matrix
    correl_matrix = covar_to_correl(covar_matrix)
    # print(f"Correl matrix:\n{correl_matrix}")
    
    # this is to get the distance matrix
    distance_matrix = 1 - np.square(correl_matrix)
    # print(f"Distance matrix:\n{distance_matrix}")
    
    matrices = [covar_matrix, correl_matrix, distance_matrix]
    for matrix in matrices:
        if check_symmetric(matrix) == False:
            print("This matrix is not symmetrical.")
            print(counter)

/tmp/ipykernel_16150/651759837.py:23: RuntimeWarning: invalid value encountered in divide
  correlation = cov_matrix / outer_v


Note: I realized that the matrices will only not be symmetrical if the window size is not a multiple of the rows. In example, if you generate a 100-row dataset, but only have a window size of 3, then it will lead to a matrix that is not symmetrical because 100 is not divisible by 3. 

### This is for a 2609 x 300 numpy array

In [17]:
np.random.seed(42)
data = np.random.randn(2609, 300)

np.random.seed(42)
nan_mask = np.random.rand(2609, 300) < 0.1  # 10% missing values
data[nan_mask] = np.nan
data

array([[ 0.49671415, -0.1382643 ,  0.64768854, ...,  0.30729952,
         0.81286212,         nan],
       [        nan, -0.56018104,  0.74729361, ..., -0.97876372,
        -0.44429326,         nan],
       [ 0.75698862, -0.92216532,  0.86960592, ...,  0.14671369,
                nan,         nan],
       ...,
       [ 0.17847475,  0.23949974,  0.37276212, ...,  0.59070092,
         1.09501415, -0.50521937],
       [ 1.14336084,  0.77809928, -1.02127259, ..., -0.01363353,
        -0.230399  , -0.84795407],
       [-1.42741373,  1.57230576,  1.67892859, ...,  0.83199905,
                nan,  0.00707005]])

In [18]:
skips = 300 # should be the number of columns
window = 110
counter = 0

for i in range(len(data)):
    # this is to get the matrix
    returns = data[i:i+window]

    # this is to get the covar matrix
    covar_matrix = returns_to_covar(returns)
    counter += 1
    # print(f"Covar matrix:\n{covar_matrix}")
    
    # this is to get the correl matrix
    correl_matrix = covar_to_correl(covar_matrix)
    # print(f"Correl matrix:\n{correl_matrix}")
    
    # this is to get the distance matrix
    distance_matrix = 1 - np.square(correl_matrix)
    # print(f"Distance matrix:\n{distance_matrix}")
    
    matrices = [covar_matrix, correl_matrix, distance_matrix]
    for matrix in matrices:
        if check_symmetric(matrix) == False:
            print(f"{matrix} is not symmetrical.")
            print(counter)

KeyboardInterrupt: 

## all stocks in the PSE

In [20]:
PSE_data = pd.read_csv(
    "daily.csv"
)
PSE_data['date_id'] = pd.to_datetime(PSE_data['date_id'])
PSE_data = PSE_data.set_index("date_id")
df_filtered = PSE_data.drop(columns=[col for col in PSE_data.columns if not (col.startswith('PH_') and col.endswith('_P')) and col != 'date_id'])

df_filtered = df_filtered.loc['2013-06-25':'2023-09-26']

def has_high_null_percentage(column, null_threshold):
    return column.isnull().mean() > null_threshold

null_percentage_threshold = round(df_filtered.shape[0] * 0.85)
columns_to_drop = []

for col in df_filtered.columns:
    if col == "date_id":  
        continue
    if has_high_null_percentage(df_filtered[col], null_percentage_threshold):
        columns_to_drop.append(col)

df_filtered = df_filtered.drop(columns=columns_to_drop)

PSE_log_returns = df_filtered.copy()
numeric_cols = PSE_log_returns.columns.difference(["date_id"])
PSE_log_returns[numeric_cols] = PSE_log_returns[numeric_cols].replace(0, np.nan)
PSE_log_returns[numeric_cols] = np.log(PSE_log_returns[numeric_cols] / PSE_log_returns[numeric_cols].shift(1))

PSE_log_returns = PSE_log_returns.loc['2013-06-26':'2023-09-26']
PSE_log_returns.columns = PSE_log_returns.columns.str.replace(r"_P", "").str.replace(r"PH_", "")

returns_df = PSE_log_returns.copy()
PSE_log_returns = PSE_log_returns.to_numpy()

In [32]:
skips = len(returns_df.columns)
window = 110
counter = 0

for i in range(len(returns_df.index)):
    # this is to get the matrix
    returns = PSE_log_returns[i:i+window]

    # this is to get the covar matrix
    covar_matrix = returns_to_covar(returns)
    counter += 1
    # print(f"Covar matrix:\n{covar_matrix}")
    
    # this is to get the correl matrix
    correl_matrix = covar_to_correl(covar_matrix)
    # print(f"Correl matrix:\n{correl_matrix}")
    
    # this is to get the distance matrix
    distance_matrix = 1 - np.square(correl_matrix)
    # print(f"Distance matrix:\n{distance_matrix}")
    
    matrices = [covar_matrix, correl_matrix, distance_matrix]
    for matrix in matrices:
        if check_symmetric(matrix) == True:
            print("This matrix is symmetrical.")
            print(counter)
        
        # if check_symmetric(matrix) == False:
        #     print("This matrix is not symmetrical.")
        # print(counter)

/tmp/ipykernel_16150/2711071199.py:10: RuntimeWarning: invalid value encountered in scalar divide
  test[i, j] = np.sum((pi - np.mean(pi))*(pj - np.mean(pj)))/len(pi)
/tmp/ipykernel_16150/2711071199.py:23: RuntimeWarning: invalid value encountered in divide
  correlation = cov_matrix / outer_v
